In [4]:
#%% [Cell 1] - INSTALL REQUIRED LIBRARIES & CHECK GPU
!pip install xgboost ultralytics pillow --quiet

import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

# Create a MirroredStrategy to automatically use multiple GPUs (e.g., T4 ×2)
strategy = tf.distribute.MirroredStrategy()
print("Strategy:", strategy)


Num GPUs Available: 2
Strategy: <tensorflow.python.distribute.mirrored_strategy.MirroredStrategy object at 0x7b1706021bd0>


In [5]:
import os
import shutil
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras import layers, models
from sklearn.utils.class_weight import compute_class_weight
from collections import Counter

print("✅ Dependencies Loaded!")


✅ Dependencies Loaded!


In [6]:
# Enable Mirrored Strategy for multi-GPU training
strategy = tf.distribute.MirroredStrategy()
print(f"Num GPUs Available: {strategy.num_replicas_in_sync}")


Num GPUs Available: 2


In [7]:
# Define dataset path in Kaggle
DATASET_PATH = "/kaggle/input/cow-health-dataset/cow_health_dataset2/Pink Eye(DONE)/cow_health_dataset2"

# Check dataset structure
print("Classes in Dataset:", os.listdir(DATASET_PATH))


Classes in Dataset: ['Photosensitization(DONE)', 'Bovine Warts(DONE)', 'Digital Dermatitis(also causes lameness)', 'Mange(DONE)', 'Mastititis(DONE)', 'Foot and Mouth Disease(DONE )', 'healthycows(DONE)', 'Hoof Rot(DONE)', 'Pediculosis(DONE)', 'Dermatophytosis(DONE)', 'Lumpy Skin Diseases(DONE)', 'Actinomycosis(DONE)', 'Abscess(DONE)', 'Pink Eye(DONE)', 'Bovine spongiform encephalopathy (BSE)', 'Bovine Dermatophilosis (Rain Rot)(DONE)']


In [8]:
# Define output paths
split_base = "/kaggle/working/split_cow_health_dataset"
train_dir = os.path.join(split_base, "train")
val_dir = os.path.join(split_base, "val")
test_dir = os.path.join(split_base, "test")

# Remove existing directories if running again
if os.path.exists(split_base):
    shutil.rmtree(split_base)

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Split function
def split_and_copy(src_folder, train_dest, val_dest, test_dest, train_ratio=0.7, val_ratio=0.15):
    class_counts = {}  

    for category in os.listdir(src_folder):
        category_path = os.path.join(src_folder, category)
        if not os.path.isdir(category_path):
            continue

        images = os.listdir(category_path)
        random.shuffle(images)

        total = len(images)
        train_split = int(total * train_ratio)
        val_split = int(total * (train_ratio + val_ratio))

        train_images = images[:train_split]
        val_images = images[train_split:val_split]
        test_images = images[val_split:]

        os.makedirs(os.path.join(train_dest, category), exist_ok=True)
        os.makedirs(os.path.join(val_dest, category), exist_ok=True)
        os.makedirs(os.path.join(test_dest, category), exist_ok=True)

        for img in train_images:
            shutil.copy(os.path.join(category_path, img), os.path.join(train_dest, category, img))

        for img in val_images:
            shutil.copy(os.path.join(category_path, img), os.path.join(val_dest, category, img))

        for img in test_images:
            shutil.copy(os.path.join(category_path, img), os.path.join(test_dest, category, img))

        class_counts[category] = total 

    print("✅ Dataset split complete!")
    return class_counts

class_counts = split_and_copy(DATASET_PATH, train_dir, val_dir, test_dir)


✅ Dataset split complete!


In [10]:
import os

# Define the base path where the split dataset is stored
base_path = "/kaggle/working/split_cow_health_dataset"

# Function to print folder structure
def print_folder_structure(base_path):
    for split in ['train', 'val', 'test']:
        split_path = os.path.join(base_path, split)
        if not os.path.exists(split_path):
            print(f"🚨 {split} folder not found!")
            continue

        print(f"\n📂 {split.upper()} Directory:")
        for class_folder in sorted(os.listdir(split_path)):
            class_path = os.path.join(split_path, class_folder)
            if os.path.isdir(class_path):
                num_files = len(os.listdir(class_path))
                print(f"  📁 {class_folder} - {num_files} images")

# Run the function
print_folder_structure(base_path)



📂 TRAIN Directory:
  📁 Abscess(DONE) - 14 images
  📁 Actinomycosis(DONE) - 23 images
  📁 Bovine Dermatophilosis (Rain Rot)(DONE) - 88 images
  📁 Bovine Warts(DONE) - 93 images
  📁 Bovine spongiform encephalopathy (BSE) - 21 images
  📁 Dermatophytosis(DONE) - 142 images
  📁 Digital Dermatitis(also causes lameness) - 61 images
  📁 Foot and Mouth Disease(DONE ) - 362 images
  📁 Hoof Rot(DONE) - 27 images
  📁 Lumpy Skin Diseases(DONE) - 638 images
  📁 Mange(DONE) - 28 images
  📁 Mastititis(DONE) - 270 images
  📁 Pediculosis(DONE) - 104 images
  📁 Photosensitization(DONE) - 14 images
  📁 Pink Eye(DONE) - 84 images
  📁 healthycows(DONE) - 1355 images

📂 VAL Directory:
  📁 Abscess(DONE) - 3 images
  📁 Actinomycosis(DONE) - 5 images
  📁 Bovine Dermatophilosis (Rain Rot)(DONE) - 19 images
  📁 Bovine Warts(DONE) - 20 images
  📁 Bovine spongiform encephalopathy (BSE) - 5 images
  📁 Dermatophytosis(DONE) - 30 images
  📁 Digital Dermatitis(also causes lameness) - 13 images
  📁 Foot and Mouth Disea

In [11]:
class_labels = list(class_counts.keys())
class_samples = np.array(list(class_counts.values()))

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.arange(len(class_labels)),
    y=np.concatenate([[i] * n for i, n in enumerate(class_samples)])
)

class_weight_dict = {i: class_weights[i] for i in range(len(class_labels))}
print("Computed Class Weights:", class_weight_dict)


Computed Class Weights: {0: 14.8625, 1: 2.218283582089552, 2: 3.377840909090909, 3: 7.43125, 4: 0.7680878552971576, 5: 0.5738416988416989, 6: 0.15353822314049587, 7: 7.621794871794871, 8: 1.99496644295302, 9: 1.4642857142857142, 10: 0.32593201754385964, 11: 9.007575757575758, 12: 14.8625, 13: 2.4770833333333333, 14: 9.588709677419354, 15: 2.359126984126984}


In [19]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest',
)


val_test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="categorical"
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="categorical"
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="categorical"
)


Found 3324 images belonging to 16 classes.
Found 713 images belonging to 16 classes.
Found 719 images belonging to 16 classes.


In [13]:
from tensorflow.keras.optimizers import Adam


In [22]:
from tensorflow.keras.applications import MobileNetV3Small, EfficientNetB3
from tensorflow.keras import layers, models, optimizers, regularizers, callbacks

with strategy.scope():
    # MobileNetV3 Small Model
    base_model = MobileNetV3Small(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
    
    # Fine-tune last few layers
    for layer in base_model.layers[-40:]:
        layer.trainable = True

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        layers.Dense(512, activation="relu", kernel_regularizer=regularizers.l2(0.01)),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        layers.Dense(len(class_labels), activation="softmax")
    ])

    # Learning Rate Scheduler
    lr_scheduler = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6)

    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.0005),  # Slightly reduced LR for stability
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

model.summary()

# Train the model with augmentation
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,  # Increased epochs for better convergence
    class_weight=class_weight_dict,
    callbacks=[lr_scheduler]
)

# Save the improved MobileNetV3 model
model.save("cow_health_mobilenetv3_tuned.h5")


# EfficientNetB3 Model
with strategy.scope():
    base_model_eff = EfficientNetB3(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

    # Fine-tune last few layers
    for layer in base_model_eff.layers[-150:]:
        layer.trainable = True

    model_eff = models.Sequential([
        base_model_eff,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        layers.Dense(512, activation="relu", kernel_regularizer=regularizers.l2(0.01)),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        layers.Dense(len(class_labels), activation="softmax")
    ])

    model_eff.compile(
        optimizer=optimizers.Adam(learning_rate=0.0005),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

model_eff.summary()

# Train EfficientNetB3 with augmentation
history_eff = model_eff.fit(
    train_generator,
    validation_data=val_generator,
    epochs=40,
    class_weight=class_weight_dict,
    callbacks=[lr_scheduler]
)

# Save the improved EfficientNetB3 model
model_eff.save("cow_health_efficientnetb3_tuned.h5")


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ MobileNetV3Small (Functional)        │ (None, 7, 7, 576)           │         939,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_5           │ (None, 576)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, 576)                 │           2,304 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 576)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 512)                 │         295,424 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_11               │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 16)                  │           8,208 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,247,104 (4.76 MB)

 Trainable params: 1,232,816 (4.70 MB)

 Non-trainable params: 14,288 (55.81 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetb3 (Functional)          │ (None, 7, 7, 1536)          │      10,783,535 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_6           │ (None, 1536)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_12               │ (None, 1536)                │           6,144 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 1536)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 512)                 │         786,944 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_13               │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 16)                  │           8,208 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,586,879 (44.20 MB)

 Trainable params: 11,495,480 (43.85 MB)

 Non-trainable params: 91,399 (357.03 KB)

Epoch 1/40
104/104 ━━━━━━━━━━━━━━━━━━━━ 142s 646ms/step - accuracy: 0.2938 - loss: 18.4578 - val_accuracy: 0.1120 - val_loss: 10.5272 - learning_rate: 5.0000e-04
Epoch 2/40
104/104 ━━━━━━━━━━━━━━━━━━━━ 63s 571ms/step - accuracy: 0.5837 - loss: 12.7342 - val_accuracy: 0.0952 - val_loss: 10.0552 - learning_rate: 5.0000e-04
Epoch 3/40
104/104 ━━━━━━━━━━━━━━━━━━━━ 63s 575ms/step - accuracy: 0.6515 - loss: 11.2781 - val_accuracy: 0.0700 - val_loss: 11.0262 - learning_rate: 5.0000e-04
Epoch 4/40
104/104 ━━━━━━━━━━━━━━━━━━━━ 63s 572ms/step - accuracy: 0.6847 - loss: 10.3870 - val_accuracy: 0.2185 - val_loss: 10.1213 - learning_rate: 5.0000e-04
Epoch 5/40
104/104 ━━━━━━━━━━━━━━━━━━━━ 62s 568ms/step - accuracy: 0.7011 - loss: 9.5285 - val_accuracy: 0.5742 - val_loss: 8.0607 - learning_rate: 5.0000e-04
Epoch 6/40
104/104 ━━━━━━━━━━━━━━━━━━━━ 62s 563ms/step - accuracy: 0.7454 - loss: 8.7855 - val_accuracy: 0.6246 - val_loss: 7.5090 - learning_rate: 5.0000e-04
Epoch 7/40
104/104 ━━━━━━━━━━━━━━━━━━

Alternate

In [23]:
# Evaluate MobileNetV3 Small on Test Data
print("\nEvaluating MobileNetV3 Small on Test Data...")
test_loss_mobilenet, test_acc_mobilenet = model.evaluate(test_generator)
print(f"MobileNetV3 Small - Test Accuracy: {test_acc_mobilenet:.4f}, Test Loss: {test_loss_mobilenet:.4f}")

# Evaluate EfficientNetB3 on Test Data
print("\nEvaluating EfficientNetB3 on Test Data...")
test_loss_eff, test_acc_eff = model_eff.evaluate(test_generator)
print(f"EfficientNetB3 - Test Accuracy: {test_acc_eff:.4f}, Test Loss: {test_loss_eff:.4f}")



Evaluating MobileNetV3 Small on Test Data...
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - accuracy: 0.0901 - loss: 9.8096
MobileNetV3 Small - Test Accuracy: 0.0667, Test Loss: 9.7515

Evaluating EfficientNetB3 on Test Data...
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step - accuracy: 0.8225 - loss: 1.5142
EfficientNetB3 - Test Accuracy: 0.8306, Test Loss: 1.5573


In [25]:
import shutil

# Move the saved models to the output directory
shutil.move("cow_health_mobilenetv3_tuned.h5", "/kaggle/working/cow_health_mobilenetv3_tuned.h5")
shutil.move("cow_health_efficientnetb3_tuned.h5", "/kaggle/working/cow_health_efficientnetb3_tuned.h5")


'/kaggle/working/cow_health_efficientnetb3_tuned.h5'

In [26]:
from IPython.display import FileLink

# Generate a clickable link
FileLink("cow_health_efficientnetb3_tuned.h5")


/kaggle/working/cow_health_efficientnetb3_tuned.h5

***REDUNDANT CODE***


In [24]:
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras import layers, models, optimizers, regularizers, callbacks
import tensorflow as tf

# MobileNetV3 Large Model with Fine-Tuning
with strategy.scope():
    base_model = MobileNetV3Large(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
    
    # Freeze all layers initially
    base_model.trainable = False

    # Unfreeze only the last 40 layers for fine-tuning
    for layer in base_model.layers[-40:]:  # Adjust this number based on experimentation
        layer.trainable = True

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        layers.Dense(512, activation="relu", kernel_regularizer=regularizers.l2(0.01)),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        layers.Dense(len(class_labels), activation="softmax")
    ])

    # Learning Rate Scheduler
    lr_scheduler = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6)

    # Compile with a low learning rate for stable fine-tuning
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-5),  
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

model.summary()

# Train MobileNetV3 Large
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    class_weight=class_weight_dict,
    callbacks=[lr_scheduler]
)

# Save the fine-tuned MobileNetV3 Large model
model.save("cow_health_mobilenetv3large_finetuned.h5")


12683000/12683000 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ MobileNetV3Large (Functional)        │ (None, 7, 7, 960)           │       2,996,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_7           │ (None, 960)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_14               │ (None, 960)                 │           3,840 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 960)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 512)                 │         492,032 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_15               │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_15 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 16)                  │           8,208 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,502,480 (13.36 MB)

 Trainable params: 2,587,896 (9.87 MB)

 Non-trainable params: 914,584 (3.49 MB)

Epoch 1/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 64s 455ms/step - accuracy: 0.0861 - loss: 22.2285 - val_accuracy: 0.0084 - val_loss: 9.6347 - learning_rate: 1.0000e-05
Epoch 2/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 47s 418ms/step - accuracy: 0.0965 - loss: 21.5622 - val_accuracy: 0.0056 - val_loss: 9.5836 - learning_rate: 1.0000e-05
Epoch 3/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 47s 421ms/step - accuracy: 0.0956 - loss: 20.5452 - val_accuracy: 0.0028 - val_loss: 9.5981 - learning_rate: 1.0000e-05
Epoch 4/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 47s 420ms/step - accuracy: 0.1209 - loss: 20.1414 - val_accuracy: 0.0952 - val_loss: 9.6094 - learning_rate: 1.0000e-05
Epoch 5/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 47s 424ms/step - accuracy: 0.1152 - loss: 20.0809 - val_accuracy: 0.0700 - val_loss: 9.6778 - learning_rate: 1.0000e-05
Epoch 6/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 47s 418ms/step - accuracy: 0.1382 - loss: 19.5543 - val_accuracy: 0.0840 - val_loss: 9.7160 - learning_rate: 5.0000e-06
Epoch 7/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 

Mobilenetv3 large

Mobilevit

EfficientDET

In [ ]:
evaluate models on test set

In [14]:
test_loss, test_acc = model.evaluate(val_generator)
print(f"✅ Test Accuracy: {test_acc * 100:.2f}%")


23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - accuracy: 0.3798 - loss: 2.1512
✅ Test Accuracy: 38.94%


✅ Expected Output:

6 images from the test set

Predicted class for each image



In [15]:
def predict_and_display(model, test_data):
    predictions = model.predict(test_data)
    predicted_classes = np.argmax(predictions, axis=1)

    # Display some images with predictions
    plt.figure(figsize=(10, 6))
    for i in range(6):
        plt.subplot(2, 3, i+1)
        plt.imshow(test_data[i][0])
        plt.title(f"Predicted: {list(test_data.class_indices.keys())[predicted_classes[i]]}")
        plt.axis('off')
    plt.show()

print("MobileNetV3 Predictions:")
predict_and_display(mobilenetv3_large, test_data)

print("MobileViT Predictions:")
predict_and_display(mobilevit, test_data)

print("EfficientDet Predictions:")
predict_and_display(efficientnet, test_data)


MobileNetV3 Predictions:


NameError: name 'mobilenetv3_large' is not defined

In [ ]:
MODEL_PATH = "/kaggle/working/mobilenetv3_cow_health.h5"
model.save(MODEL_PATH)
print(f"✅ Model saved at {MODEL_PATH}")


In [23]:
#%% [Cell 2] - UPDATED DATASET SPLITTING WITH RECURSIVE SEARCH
import os
import shutil
import random

# Set your dataset paths
dataset_source = "/kaggle/input/cow-health-dataset"
dataset_dest = "/kaggle/working/cow_health_dataset_split"

# List of valid image file extensions (you can update if needed)
IMAGE_EXTENSIONS = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff'}

def is_image_file(filename):
    return os.path.splitext(filename)[1].lower() in IMAGE_EXTENSIONS

def split_dataset(source_dir, dest_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    """
    Splits dataset from source_dir into train, val, and test folders.
    This function recursively finds image files in nested subdirectories.
    Assumes that the immediate subdirectory of source_dir is the class name.
    """
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    for split in ['train', 'val', 'test']:
        split_path = os.path.join(dest_dir, split)
        if not os.path.exists(split_path):
            os.makedirs(split_path)
    
    # Iterate over each class folder in the source directory.
    classes = [d for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d))]
    for cls in classes:
        cls_dir = os.path.join(source_dir, cls)
        # Recursively collect image file paths from nested subdirectories.
        images = []
        for root, dirs, files in os.walk(cls_dir):
            for file in files:
                if is_image_file(file):
                    images.append(os.path.join(root, file))
                    
        random.shuffle(images)
        n = len(images)
        train_end = int(n * train_ratio)
        val_end = train_end + int(n * val_ratio)
        
        for split, img_list in zip(['train', 'val', 'test'], 
                                   [images[:train_end], images[train_end:val_end], images[val_end:]]):
            split_cls_path = os.path.join(dest_dir, split, cls)
            os.makedirs(split_cls_path, exist_ok=True)
            for img_path in img_list:
                # Copy each image file to the destination
                dst = os.path.join(split_cls_path, os.path.basename(img_path))
                shutil.copy(img_path, dst)
    print("Dataset splitting complete.")

# Run the dataset split (only run once; comment out later if needed)
split_dataset(dataset_source, dataset_dest)


Dataset splitting complete.


In [25]:
import os
import shutil
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV3Large, MobileNetV3Small
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Set dataset paths
dataset_source = "/kaggle/input/cow-health-dataset"
dataset_dest = "/kaggle/working/cow_health_dataset_split"

# List of valid image file extensions
IMAGE_EXTENSIONS = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff'}

def is_image_file(filename):
    return os.path.splitext(filename)[1].lower() in IMAGE_EXTENSIONS

def get_all_images(cls_dir):
    """ Recursively get all image files from a class directory. """
    images = []
    for root, _, files in os.walk(cls_dir):
        for file in files:
            if is_image_file(file):
                images.append(os.path.join(root, file))
    return images

def split_dataset(source_dir, dest_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    """
    Splits dataset into train, val, and test folders.
    Ensures proper dataset distribution without data leakage.
    """
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    
    for split in ['train', 'val', 'test']:
        os.makedirs(os.path.join(dest_dir, split), exist_ok=True)
    
    classes = [d for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d))]
    
    for cls in classes:
        cls_dir = os.path.join(source_dir, cls)
        images = get_all_images(cls_dir)
        
        random.shuffle(images)
        n = len(images)
        train_end = int(n * train_ratio)
        val_end = train_end + int(n * val_ratio)
        
        for split, img_list in zip(['train', 'val', 'test'],
                                   [images[:train_end], images[train_end:val_end], images[val_end:]]):
            split_cls_path = os.path.join(dest_dir, split, cls)
            os.makedirs(split_cls_path, exist_ok=True)
            for img_path in img_list:
                shutil.copy2(img_path, os.path.join(split_cls_path, os.path.basename(img_path)))
    print("Dataset splitting complete.")

# Run dataset split (only run once)
split_dataset(dataset_source, dataset_dest)

# Data augmentation (only for training)
IMG_SIZE = (224, 224)
BATCH_SIZE = 16

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    os.path.join(dataset_dest, 'train'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = test_datagen.flow_from_directory(
    os.path.join(dataset_dest, 'val'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    os.path.join(dataset_dest, 'test'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Define MobileNetV3 Model
def create_mobilenetv3_model(input_shape, num_classes):
    base_model = MobileNetV3Large(weights='imagenet', include_top=False, input_shape=input_shape)
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dense(128, activation='relu')(x)
    output = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs=base_model.input, outputs=output)
    return model

# Compile model with MirroredStrategy
strategy = tf.distribute.MirroredStrategy()
print("Using strategy:", strategy)

with strategy.scope():
    num_classes = len(train_generator.class_indices)
    input_shape = IMG_SIZE + (3,)
    
    model_mnv3 = create_mobilenetv3_model(input_shape, num_classes)
    
    model_mnv3.compile(
        optimizer=optimizers.Adam(learning_rate=1e-4),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

# Train MobileNetV3 Model
history = model_mnv3.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    verbose=1
)

# Save Model
model_mnv3.save("/kaggle/working/mobilenetv3_cow_health.h5")
print("Training complete! Model saved.")


KeyboardInterrupt: 

In [20]:
#%% [Cell 3] - DATA AUGMENTATION & DATA GENERATORS
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input

from sklearn.utils.class_weight import compute_class_weight

IMG_SIZE = (224, 224)  # Adjust for faster training on GPU
BATCH_SIZE = 16        # Adjust based on GPU memory

# Heavy augmentation for training to help with imbalance
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Validation and test: only preprocessing
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    os.path.join(dataset_dest, 'train'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = test_datagen.flow_from_directory(
    os.path.join(dataset_dest, 'val'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    os.path.join(dataset_dest, 'test'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)



Found 3606 images belonging to 1 classes.
Found 1191 images belonging to 1 classes.
Found 1197 images belonging to 1 classes.


In [6]:
#%% [Cell 4] - YOLOv8S FOR REGION OF INTEREST EXTRACTION (OPTIONAL)
# YOLO can help crop to relevant areas like lesions or wounds.
# Make sure 'ultralytics' is installed. 
from ultralytics import YOLO
import tensorflow as tf

# Load YOLOv8s model (downloads weights on first run if needed)
yolo_model = YOLO("yolov8s.pt")

def detect_regions(image_path):
    """
    Uses YOLOv8s to detect regions in an image.
    Returns a cropped and resized image (IMG_SIZE) based on the first detection,
    or the resized full image if no detection.
    """
    results = yolo_model(image_path, verbose=False)
    boxes = results[0].boxes.xyxy.cpu().numpy() if results[0].boxes.xyxy.numel() > 0 else None
    img = tf.keras.preprocessing.image.load_img(image_path)
    img = tf.keras.preprocessing.image.img_to_array(img)
    if boxes is not None:
        x1, y1, x2, y2 = boxes[0]
        cropped = img[int(y1):int(y2), int(x1):int(x2), :]
        return tf.image.resize(cropped, IMG_SIZE)
    else:
        return tf.image.resize(img, IMG_SIZE)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 21.5M/21.5M [00:00<00:00, 187MB/s]


In [8]:
#%% [Cell 5] - DEFINE DEEP LEARNING MODELS
from tensorflow.keras import layers, models, optimizers

def create_mobilenetv3_model(input_shape, num_classes):
    base_model = MobileNetV3Large(weights='imagenet', include_top=False, input_shape=input_shape)
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dense(128, activation='relu')(x)
    output = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs=base_model.input, outputs=output)
    return model

def create_mobilevit_model(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    # Simple MobileViT-like block
    x = layers.Conv2D(32, (3,3), padding='same', activation='relu')(inputs)
    x = layers.MaxPooling2D()(x)
    # Minimal Transformer-like block
    b, h, w, c = x.shape
    x_flat = layers.Reshape((h*w, c))(x)
    x_flat = layers.LayerNormalization()(x_flat)
    x_flat = layers.MultiHeadAttention(num_heads=4, key_dim=32)(x_flat, x_flat)
    x = layers.Reshape((h, w, c))(x_flat)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs, outputs)
    return model


In [9]:
#%% [Cell 6] - BUILD & COMPILE MODELS WITH MirroredStrategy
with strategy.scope():
    num_classes = len(train_generator.class_indices)
    input_shape = IMG_SIZE + (3,)

    model_mnv3 = create_mobilenetv3_model(input_shape, num_classes)
    model_mvit = create_mobilevit_model(input_shape, num_classes)

    # Compile
    model_mnv3.compile(
        optimizer=optimizers.Adam(learning_rate=1e-4),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    model_mvit.compile(
        optimizer=optimizers.Adam(learning_rate=1e-4),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

print("Models built and compiled inside MirroredStrategy scope.")


12683000/12683000 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Models built and compiled inside MirroredStrategy scope.


In [13]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# ✅ Ensure MirroredStrategy is defined only once
if "strategy" not in globals():
    strategy = tf.distribute.MirroredStrategy()

# ✅ Compute Class Weights (Inverse Frequency)
from sklearn.utils.class_weight import compute_class_weight

class_labels = [
    "Healthy", "Pediculosis", "Pink Eye", "Photosensitization", "Dermaphytosis", 
    "FMD", "Hoof Rot", "Lumpy", "Mastitis", "Abscesses", "Actinomycosis", 
    "Rain Rot", "BSE", "Warts", "Dermatitis", "Mange"
]

class_counts = np.array([
    1938, 154, 120, 20, 204, 523, 39, 912, 388, 22, 34, 132, 32, 134, 88, 40
])

total_samples = np.sum(class_counts)
class_weights = {i: total_samples / (len(class_labels) * class_counts[i]) for i in range(len(class_labels))}

print("📊 Adjusted Class Weights:", class_weights)

# ✅ Apply Data Augmentation to Boost Rare Classes
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_datagen = ImageDataGenerator(rescale=1.0 / 255)  # No augmentation for validation

# ✅ Load Train and Validation Data
train_generator = train_datagen.flow_from_directory(
    "/kaggle/working/cow_health_dataset_split/train",  # Updated path
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

val_generator = val_datagen.flow_from_directory(
    "/kaggle/working/cow_health_dataset_split/val",  # Updated path
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)


# ✅ Define EarlyStopping Callback
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# ✅ Everything inside ONE strategy.scope()
with strategy.scope():
    print("🚀 Training MobileNetV3 on T4 ×2...")
    history_mnv3 = model_mnv3.fit(
        train_generator,
        epochs=30,  # Reduce to prevent overfitting
        validation_data=val_generator,
        class_weight=class_weights,
        callbacks=[early_stop]
    )

    print("🚀 Training MobileViT on T4 ×2...")
    history_mvit = model_mvit.fit(
        train_generator,
        epochs=20,  # Reduce slightly
        validation_data=val_generator,
        class_weight=class_weights,
        callbacks=[early_stop]
    )

print("✅ Training completed successfully on T4 ×2 GPUs!")


📊 Adjusted Class Weights: {0: 0.15415376676986584, 1: 1.9399350649350648, 2: 2.4895833333333335, 3: 14.9375, 4: 1.4644607843137254, 5: 0.5712237093690249, 6: 7.660256410256411, 7: 0.3275767543859649, 8: 0.7699742268041238, 9: 13.579545454545455, 10: 8.786764705882353, 11: 2.2632575757575757, 12: 9.3359375, 13: 2.2294776119402986, 14: 3.3948863636363638, 15: 7.46875}
Found 3606 images belonging to 1 classes.
Found 1191 images belonging to 1 classes.
🚀 Training MobileNetV3 on T4 ×2...
Epoch 1/30
113/113 ━━━━━━━━━━━━━━━━━━━━ 77s 488ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/30
 61/113 ━━━━━━━━━━━━━━━━━━━━ 22s 434ms/step - accuracy: 1.0000 - loss: 0.0000e+00

KeyboardInterrupt: 

In [14]:
from tensorflow.keras.callbacks import EarlyStopping

# Define Early Stopping
early_stop = EarlyStopping(
    monitor='val_loss',  # Stop if validation loss stops improving
    patience=5,          # Wait for 5 epochs before stopping
    restore_best_weights=True  # Restore best weights
)

# Define Optimizer (RMSprop instead of Adam)
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001)

# Compile models with RMSprop
with strategy.scope():
    model_mnv3.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    model_mvit.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train models with EarlyStopping
with strategy.scope():
    print("Training MobileNetV3 on T4 ×2...")
    history_mnv3 = model_mnv3.fit(
        train_generator,
        epochs=50,  
        validation_data=val_generator,
        class_weight=class_weights,
        callbacks=[early_stop]  # Apply Early Stopping
    )

    print("Training MobileViT on T4 ×2...")
    history_mvit = model_mvit.fit(
        train_generator,
        epochs=50,  
        validation_data=val_generator,
        class_weight=class_weights,
        callbacks=[early_stop]  # Apply Early Stopping
    )

print("✅ Training completed successfully!")


Training MobileNetV3 on T4 ×2...


/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (16, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/losses/losses.py:27: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(16, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


RuntimeError: Mixing different tf.distribute.Strategy objects: <tensorflow.python.distribute.mirrored_strategy.MirroredStrategy object at 0x7c5364e3a6e0> is not <tensorflow.python.distribute.distribute_lib._DefaultDistributionStrategy object at 0x7c52fa0b30d0>

In [15]:
#%% [Cell 7] - TRAIN THE MODELS
print("Training MobileNetV3 on T4 ×2...")
history_mnv3 = model_mnv3.fit(
    train_generator,
    epochs=20,  # Adjust as needed
    validation_data=val_generator,
    class_weight=class_weights,
    
)

print("Training MobileViT on T4 ×2...")
history_mvit = model_mvit.fit(
    train_generator,
    epochs=20,  # Adjust as needed
    validation_data=val_generator,
    class_weight=class_weights,
    
)


Training MobileNetV3 on T4 ×2...
Epoch 1/20


StagingError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/optimizer.py", line 135, in _distributed_tf_update_step  **
        distribution.extended.update(
    File "/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/optimizer.py", line 132, in apply_grad_to_update_var  **
        return self.update_step(grad, var, learning_rate)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/rmsprop.py", line 122, in update_step
        velocity = self._velocities[self._get_variable_index(variable)]

    IndexError: list index out of range


In [ ]:
#%% [Cell 8] - ENSEMBLE PREDICTION
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def ensemble_predict(image):
    """
    For a preprocessed image (shape IMG_SIZE+(3,)),
    get predictions from both models and average their softmax outputs.
    """
    img_batch = np.expand_dims(image, axis=0)
    pred_mnv3 = model_mnv3.predict(img_batch)
    pred_mvit = model_mvit.predict(img_batch)
    ensemble_pred = (pred_mnv3 + pred_mvit) / 2.0
    return ensemble_pred

# Example usage
test_image_path = "/kaggle/input/cow-health-dataset/some_class/example.jpg"  # update path
img = load_img(test_image_path, target_size=IMG_SIZE)
img_array = img_to_array(img)
img_array = preprocess_input(img_array)

prediction = ensemble_predict(img_array)
predicted_class = np.argmax(prediction, axis=1)
print("Predicted class index:", predicted_class)


In [ ]:
#%% [Cell 9] - SAVE MODELS
model_mnv3.save("mobilenetv3_cow_health.h5")
model_mvit.save("mobilevit_cow_health.h5")
print("Models saved in /kaggle/working")


In [12]:
#%% [Cell] - MobileNetV3Small Model Training
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Load MobileNetV3Small (pretrained on ImageNet) without the top classification layer
base_model = MobileNetV3Small(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers (optional: if fine-tuning later, unfreeze some layers)
base_model.trainable = False  

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
x = Dense(64, activation="relu")(x)
output_layer = Dense(num_classes, activation="softmax")(x)  # Adjust 'num_classes' based on your dataset

# Compile the model
model = Model(inputs=base_model.input, outputs=output_layer)
model.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

# Summary of the model
model.summary()

# Train the model (update train_generator and val_generator based on your setup)
history = model.fit(train_generator, validation_data=val_generator, epochs=10, verbose=1)

# Save the trained model
model.save("/kaggle/working/mobilenetv3small_cow_health.h5")

print("MobileNetV3Small training complete and model saved!")


4334752/4334752 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling_1 (Rescaling)   │ (None, 224, 224, 3)    │              0 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv (Conv2D)             │ (None, 112, 112, 16)   │            432 │ rescaling_1[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv_bn                   │ (None, 112, 112, 16)   │             64 │ conv[0][0]             │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_20             │ (None, 112, 112, 16)   │              0 │ conv_bn[0][0]          │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 113, 113, 16)   │              0 │ activation_20[0][0]    │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 56, 56, 16)     │            144 │ expanded_conv_depthwi… │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 56, 56, 16)     │             64 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_19 (ReLU)           │ (None, 56, 56, 16)     │              0 │ expanded_conv_depthwi… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_squeeze_ex… │ (None, 1, 1, 16)       │              0 │ re_lu_19[0][0]         │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_squeeze_ex… │ (None, 1, 1, 8)        │            136 │ expanded_conv_squeeze… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_squeeze_ex… │ (None, 1, 1, 8)        │              0 │ expanded_conv_squeeze… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_squeeze_ex… │ (None, 1, 1, 16)       │            144 │ expanded_conv_squeeze… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_8 (Add)               │ (None, 1, 1, 16)       │              0 │ expanded_conv_squeeze… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_20 (ReLU)      

 Total params: 1,021,297 (3.90 MB)

 Trainable params: 82,177 (321.00 KB)

 Non-trainable params: 939,120 (3.58 MB)

Epoch 1/10
226/226 ━━━━━━━━━━━━━━━━━━━━ 69s 252ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
226/226 ━━━━━━━━━━━━━━━━━━━━ 46s 198ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10
226/226 ━━━━━━━━━━━━━━━━━━━━ 46s 199ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
226/226 ━━━━━━━━━━━━━━━━━━━━ 46s 197ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
226/226 ━━━━━━━━━━━━━━━━━━━━ 46s 198ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
226/226 ━━━━━━━━━━━━━━━━━━━━ 46s 195ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
226/226 ━━━━━━━━━━━━━━━━━━━━ 40s 171ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
226/226 ━━━━━━━━